In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [ ]:
%load_ext autoreload
%autoreload 2

from os.path import join

import numpy as np

from pyspark.sql import SQLContext

In [ ]:
path_root = "/home/tlin/notebooks/data"
path_pickle_source = join(
    path_root, "cache/two_domain/clean_data/movie")
path_pickle_target = join(
    path_root, "cache/two_domain/clean_data/book")
path_pickle_train = join(
    path_root, "cache/two_domain/split_data/train")
path_pickle_test = join(
    path_root, "cache/two_domain/split_data/test")
path_pickle_baseline_sim = join(
    path_root, "cache/two_domain/item_based_sim/base_sim")
path_pickle_extended_sim = join(
    path_root, "cache/two_domain/extend_sim/extendsim")
path_pickle_private_mapped_sim = join(
    path_root, "cache/two_domain/private_mapping/privatemap")
path_pickle_nonprivate_mapped_sim = join(
    path_root, "cache/two_domain/private_mapping/nonprivatemap")
path_pickle_userbased_alterEgo = join(
    path_root, "cache/two_domain/cross_sim/user_based_alterEgo")
path_pickle_itembased_alterEgo = join(
    path_root, "cache/two_domain/cross_sim/item_based_alterEgo")
path_pickle_alterEgo_userbased_sim = join(
    path_root, "cache/two_domain/cross_sim/targetdomain_userbased_sim")
path_pickle_alterEgo_itembased_sim = join(
    path_root, "cache/two_domain/cross_sim/targetdomain_itembased_sim")
path_pickle_private_policy_userbased_sim = join(
    path_root, "cache/two_domain/policy/policy_userbased_sim")
path_pickle_private_policy_itembased_sim = join(
    path_root, "cache/two_domain/policy/policy_itembased_sim")

In [ ]:
movieRDD = sc.pickleFile(path_pickle_source)
bookRDD = sc.pickleFile(path_pickle_target)
sourceRDD, targetRDD = movieRDD, bookRDD

In [ ]:
num_left = 0
ratio_split = 0.2
ratio_both = 0.8
seed = 666

In [ ]:
    def find_overlap_user(sourceRDD, targetRDD):
        """Find overlap users.
        return a RDD of uid.
        """
        return sourceRDD.keys().intersection(targetRDD.keys())
    
overlap_userRDD = find_overlap_user(sourceRDD, targetRDD)
overlap_userRDD_bd = sc.broadcast(overlap_userRDD.collect())

In [ ]:
    def distinguish_data(overlap_userRDD_bd, sourceRDD, targetRDD):
        """Distinguish data from overlap data to non-overlap data."""
        def in_split(iterators):
            for line in iterators:
                if line[0] in overlap_userRDD_bd.value:
                    yield line

        def out_split(iterators):
            for line in iterators:
                if line[0] not in overlap_userRDD_bd.value:
                    yield line

        overlap_sourceRDD = sourceRDD.mapPartitions(in_split)
        non_overlap_sourceRDD = sourceRDD.mapPartitions(out_split)
        overlap_targetRDD = targetRDD.mapPartitions(in_split)
        non_overlap_targetRDD = targetRDD.mapPartitions(out_split)

        return non_overlap_sourceRDD, overlap_sourceRDD, \
            non_overlap_targetRDD, overlap_targetRDD
        
non_overlap_sourceRDD, overlap_sourceRDD, \
    non_overlap_targetRDD, overlap_targetRDD = distinguish_data(
        overlap_userRDD_bd, sourceRDD, targetRDD)

In [ ]:
unionRDD = overlap_sourceRDD.union(
            overlap_targetRDD).reduceByKey(lambda a, b: a + b)
splits = unionRDD.randomSplit(
            [0.2, 0.8,
             1 - 0.2 - 0.8],
            seed=1)
splits[0].cache()

In [ ]:
    import random
    def determine_remaining(iterators):
        """split test dataset into (source, remain, remove).
        Args:
            iterators: (uid, lines), where line=[(iid, rating, time)*]
        Returns:
            uid, source data, remaining data(target), removed data(target)
            The remaining data can be empty.
        """
        def decide_remaining(target):
            remaining_dict = random.sample(target, 0)
            test_dict = [
                item for item in target if item not in remaining_dict]
            return [remaining_dict, test_dict]

        for uid, lines in iterators:
            source = [line for line in lines if "S:" in line[0]]
            target = filter(lambda line: line not in source, lines)
            remain, remove = decide_remaining(target)
            yield uid, source, remain, remove


test_part_dataRDD = splits[0].mapPartitions(determine_remaining).cache()
uid, lines = splits[0].take(1)[0]
source = [line for line in lines if "S:" in line[0]]
target = filter(lambda line: line not in source, lines)
remaining_dict = random.sample(list(target), 0)